In [6]:
@time using Printf
@time using LinearAlgebra
@time import SparseArrays
@time import GSL
@time using Arpack
#@time using GR
@time using QuadGK
@time using Roots
@time using PGFPlotsX

  0.000418 seconds (215 allocations: 19.234 KiB)
  0.000270 seconds (176 allocations: 16.203 KiB)
  0.000241 seconds (176 allocations: 16.172 KiB)
  0.000112 seconds (81 allocations: 5.969 KiB)
  0.000057 seconds (81 allocations: 5.969 KiB)
  0.000062 seconds (81 allocations: 5.969 KiB)
  0.180117 seconds (230.79 k allocations: 16.177 MiB)
  0.188639 seconds (173.42 k allocations: 12.264 MiB)


# Parameters 


+ Materials Database

 + GaAs, $\mathrm{Al_{x}Ga_{1-x}As}$

In [12]:
@time Material = Dict{Symbol,Dict}(
:GaAs=>Dict{Symbol,Any}(
        :Eg    => 1.519,
        :Eg300 => 1.426,
        :me    => 0.068,
        :mhh   => 0.450,
        :mlh   => 0.087,
        :ϵ     => 12.9,
        :α     => 8.871,
        :β     => 572.0,
        :gD    => 2,
        :gA    => 4,
        :EgT   => T -> (1.519-(((5.408E-4)*(T^2))/(T + 204)))),
    
:AlGaAs => Dict{Symbol,Any}(
                :Eg300 =>x-> 1426+1.247*x
                :me  => x -> 0.067 + 0.083*x,
                :mhh => x -> 0.50  + 0.29*x,
                :mlh => x -> 0.087 + 0.063*x,
                :ϵ   => x -> 12.9 - 3.12*x
        ));


Material[:GaAs][:Eg300]

  0.071181 seconds (48.45 k allocations: 2.934 MiB, 98.71% compilation time)


1.426

# Design Profile ($\Gamma$) 

+ Test 

In [3]:
global nm   = 1e-9
global q    = GSL.Const.MKSA_ELECTRON_CHARGE #C
global Kb   = GSL.Const.MKSA_BOLTZMANN #J/K
global Kbe  = GSL.Const.MKSA_BOLTZMANN /GSL.Const.MKSA_ELECTRON_CHARGE #eV
global ħ    = GSL.Const.MKSA_PLANCKS_CONSTANT_HBAR 
global ħeV  = GSL.Const.MKSA_PLANCKS_CONSTANT_HBAR /GSL.Const.MKSA_ELECTRON_CHARGE 
global m0   = GSL.Const.MKSA_MASS_ELECTRON#kg
global ϵ0   = GSL.GSL.Const.MKSA_VACUUM_PERMITTIVITY #F/m
# global J2eV = 1/q #Joules to eV
# global eV2J = 1*q #eV to Joules

global meV  = 1E3 
function Structure(str,T,dx,Qc,Qv)
    dx = dx*nm;
    No_Layers = size(str)[1]
    @printf "Total number of layer in the Structure: %d \n" No_Layers
    xmax = sum(str[i,2] for i in 1:No_Layers)*nm
    nmax = floor(Int,round(xmax/dx))
    xaxis = range(0, stop = xmax , length = nmax) 
    # Create arrays to export
    CB_Meff = zeros(nmax)
    VCB     = zeros(nmax)
    position = 0 
    global Δc
    
    for i in 1:No_Layers
        matType = str[i,1]
        startindex = floor(Int,round(position*nm/dx))
        position +=  str[i,2]
        finishindex = floor(Int,round(position*nm/dx))
        x = str[i,3]
        
        # fill potential profile, mass effective and dielectric vectors
        if matType == :GaAs
            if startindex == 0
                VCB[startindex+1:finishindex].=     0
                ϵ[startindex+1:finishindex].=     Material[:GaAs][:ϵ]*ϵ0
                CB_Meff[startindex+1:finishindex].= Material[:GaAs][:me]*m0
            else
                VCB[startindex:finishindex].=     0
                ϵ[startindex:finishindex].=     Material[:GaAs][:ϵ]*ϵ0
                CB_Meff[startindex:finishindex].= Material[:GaAs][:me]*m0
            end
        elseif  matType == :AlGaAs
            if startindex == 0
                VCB[startindex+1:finishindex].=     (Material[:GaAs][:Eg300])
                CB_Meff[startindex+1:finishindex].=  (Material[:AlGaAs][:me](x))*m0
            else
                VCB[startindex:finishindex].=      Δc #(GaAs[:CBO]*(x*AlAs[:Eg]+(1-x)*GaAs[:Eg]))*q
                CB_Meff[startindex:finishindex].= (Material[:AlGaAs][:me](x))*m0
            end     
        end
        
        
    end
    
    return Dict{Symbol,Any}(:VCB=>VCB,:ϵ=>ϵ,:CBMeff=>CB_Meff,:Dop=>dop,:xaxis=>xaxis,:nmax=>nmax,:Δc => Δc)
    
end;



In [9]:
Kb

1.3806504e-23

# Schrodinger Equation

 + Define Hamiltonian

In [5]:
function Hamiltonian(pot::Array,mass::Array,dx::Float64,n::Integer)
    vb = pot
    meff = mass
    H = zeros(n,n)
    dxi = dx*nm
    m_minus = (meff[1]+meff[2])/2.0
    m_plus  = (meff[1]+meff[2])/2.0
    sn_minus = -(ħ^2/dxi^2)/(2.0*m_minus)
    sn_plus  = -(ħ^2/dxi^2)/(2.0*m_plus)
    bi       = 0.5*(ħ^2/dxi^2)*((m_plus+m_minus)/(m_plus*m_minus))
    H[1,1] = bi + 0
    H[1,2] = sn_plus
    for i in 2:n-1
        m_minus  = (meff[i]+meff[i-1])/2.0
        m_plus   = (meff[i+1]+meff[i])/2.0
        sn_minus = -(ħ^2/dxi^2)/(2.0*m_minus)
        sn_plus  = -(ħ^2/dxi^2)/(2.0*m_plus)
        bi       = 0.5*(ħ^2/dxi^2)*((m_plus+m_minus)/(m_plus*m_minus))
        H[i,i-1] = sn_minus
        H[i,i]   = bi + vb[i]
        H[i,i+1] = sn_plus
    end
    m_minus = (meff[n-1]+meff[n])/2.0
    m_plus  = (meff[n]+meff[n-1])/2.0
    sn_minus = -(ħ^2/dxi^2)/(2.0*m_minus)
    sn_plus  = -(ħ^2/dxi^2)/(2.0*m_plus)
    bi = 0.5*(ħ^2/dxi^2)*((m_plus+m_minus)/(m_plus*m_minus))
    H[n,n] = bi + 0
    H[n,n-1] = sn_minus
    
    return H      
end;

 + Define $\mathrm{Schr{o}dinger}$ function
 

In [6]:
function Schrodinger(pot,mass,dx,n,sparse::Bool,subbands::Integer)
    H = Hamiltonian(pot,mass,dx,n)
    if sparse == true
        Hn  = SparseArrays.sparse(H)
        solve = eigs(Hn,nev=subbands,which=:LR,sigma=0)
        eig =  solve[1]
         ψ  =  solve[2]
    elseif sparse == false
        eig = eigvals(H)[1:subbands]
        ψ   = eigvecs(H)[:,1:subbands]
    end
    return Dict{Symbol,Array}(:λ => eig, :ψ => ψ)
end;

            

In [22]:
str = [ :AlGaAs 30.0 0.15 0 "n" :layer  ;
        :GaAs   11.87 0.0 0 "n" :well ;
        :AlGaAs 1.98  0.15 0 "n" :layer  ;
        :GaAs   11.87 0.0 0  "n" :well ;
        :AlGaAs 30.0 0.15 0 "n" :layer  ;
        ]

dx = 0.2
st = Structure(str,30,dx)


VCB     = st[:VCB];
Eps     = st[:ϵ];
CB_Meff = st[:CBMeff];
dop     = st[:Dop];
xaxis   = st[:xaxis];
nmax    = st[:nmax];
Δc      = st[:Δc];


Schrodinger(VCB,CB_Meff,dx,nmax,true,5)

Total number of layer in the Structure: 5 


Dict{Symbol,Array} with 2 entries:
  :λ => [2.54188e-21, 3.77735e-21, 1.07972e-20, 1.45843e-20, 2.07744e-20]
  :ψ => [-4.7322e-9 -1.05293e-8 … -3.53904e-6 0.0013317; -9.45814e-9 -2.1038e-8…

In [15]:
0.1*10^-9

1.0000000000000006e-10